Tabular Classification

In [ ]:
!pip install opendatasets --quiet
import opendatasets as od
od.download("https://www.kaggle.com/datasets/mssmartypants/rice-type-classification")

In [ ]:
import os
os.listdir("/content/rice-type-classification")


In [ ]:
import pandas as pd # Data reading and preprocessing


In [ ]:
data_df = pd.read_csv("/content/rice-type-classification/riceClassification.csv")
data_df.head()


In [ ]:
data_df.dropna(inplace=True)
data_df.drop(["id"], axis=1, inplace= True)  # Drop Id column # axis=0 row-wise operation # axis=1 column-wise operation       
data_df.head()

In [ ]:
	Area	MajorAxisLength	MinorAxisLength	Eccentricity	ConvexArea	EquivDiameter	Extent	Perimeter	Roundness	AspectRation	Class
0	4537	92.229316	64.012769	0.719916	4677	76.004525	0.657536	273.085	0.764510	1.440796	1
1	2872	74.691881	51.400454	0.725553	3015	60.471018	0.713009	208.317	0.831658	1.453137	1
2	3048	76.293164	52.043491	0.731211	3132	62.296341	0.759153	210.012	0.868434	1.465950	1
3	3073	77.033628	51.928487	0.738639	3157	62.551300	0.783529	210.657	0.870203	1.483456	1
4	3693	85.124785	56.374021	0.749282	3802	68.571668	0.769375	230.332	0.874743	1.510000	1


In [ ]:
Data Preprocessing

In [ ]:
original_df = data_df.copy() # Creating a copy of the original Dataframe to use to normalize inference

for column in data_df.columns:
  data_df[column] = data_df[column]/data_df[column].abs().max()
data_df.head()

In [ ]:
Area	MajorAxisLength	MinorAxisLength	Eccentricity	ConvexArea	EquivDiameter	Extent	Perimeter	Roundness	AspectRation	Class
0	0.444368	0.503404	0.775435	0.744658	0.424873	0.666610	0.741661	0.537029	0.844997	0.368316	1.0
1	0.281293	0.407681	0.622653	0.750489	0.273892	0.530370	0.804230	0.409661	0.919215	0.371471	1.0
2	0.298531	0.416421	0.630442	0.756341	0.284520	0.546380	0.856278	0.412994	0.959862	0.374747	1.0
3	0.300979	0.420463	0.629049	0.764024	0.286791	0.548616	0.883772	0.414262	0.961818	0.379222	1.0
4	0.361704	0.464626	0.682901	0.775033	0.345385	0.601418	0.867808	0.452954	0.966836	0.386007	1.0


In [ ]:
Data Splitting

In [ ]:
import numpy as np# Mathematical operations
from sklearn.model_selection import train_test_split # Split the dataset (train, validation, test)

In [ ]:
X = np.array(data_df.iloc[:,:-1])# Get the inputs, all rows and all columns except last column (output)
Y = np.array(data_df.iloc[:, -1])# Get the ouputs, all rows and last column only (output column)

In [ ]:
print("Training set is: ", X_train.shape[0])
print("Training set is: ", X_test.shape[0])
print("Training set is: ", X_val.shape[0])


Training set is:  12729
Training set is:  2728
Training set is:  2728

In [ ]:
Dataset Object

In [ ]:
mport torch # Torch main framework
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torchsummary import summary # Visualize the model layers and number of parameters

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' # detect the GPU if any, if not use CPU, change cuda to mps if you have a mac

In [ ]:
class CustomDataset(Dataset):
  def __init__(self,X, Y):
    self.X = torch.tensor(X, dtype=torch.float32).to(device)
    self.Y = torch.tensor(Y, dtype = torch.float32).to(device)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    return self.X[index], self.Y[index]


In [ ]:
training_data = CustomDataset(X_train, Y_train)
test_data = CustomDataset(X_test, Y_test)
validate_data = CustomDataset(X_val,Y_val)

In [ ]:
Data Loaders

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)
validate_data = DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
HIDDEN_LAYERS = 10

class MyModel(nn.Module):
  def __init__(self):
    super(MyModel, self).__init__()


    self.input_layer = nn.Linear(X.shape[1], HIDDEN_LAYERS)
    self.linear = nn.Linear(HIDDEN_LAYERS,1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x = self.input_layer(x)
    x = self.linear(x)
    x = self.sigmoid(x)
    return x

In [ ]:
model = MyModel().to(device)
summary(model, (X.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
================================================================
            Linear-1                   [-1, 10]             110
            Linear-2                    [-1, 1]              11
           Sigmoid-3                    [-1, 1]               0
================================================================
Total params: 121
Trainable params: 121
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------

In [ ]:
Loss and Optimizer

In [ ]:
from torch.optim import Adam # Adam Optimizer

criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr = 1e-3)

In [ ]:
Training
Now we are ready exiciting part.we are training our Model

In [ ]:
EPOCHS = 10

train_losses = []
train_accs = []
val_losses = []
val_accs = []

for epoch in range(EPOCHS):
    # -------- TRAINING ----------
    total_loss = 0
    total_acc = 0

    for inputs, labels in train_dataloader:
        optimizer.zero_grad()

        # Forward pass
        pred = model(inputs).squeeze(1)
        loss = criterion(pred, labels)

        # Backward pass and weight update
        loss.backward()
        optimizer.step()

        # Metrics
        total_loss += loss.item()
        total_acc += ((pred.round() == labels).sum().item()) / len(labels)

    avg_train_loss = total_loss / len(train_dataloader)
    avg_train_acc = total_acc / len(train_dataloader)

    train_losses.append(avg_train_loss)
    train_accs.append(avg_train_acc)

    # -------- VALIDATION ----------
    total_val_loss = 0
    total_val_acc = 0

    with torch.no_grad():
        for inputs, labels in validate_data:
            pred = model(inputs).squeeze(1)
            loss = criterion(pred, labels)

            total_val_loss += loss.item()
            total_val_acc += ((pred.round() == labels).sum().item()) / len(labels)

    avg_val_loss = total_val_loss / len(validate_data)
    avg_val_acc = total_val_acc / len(validate_data)

    val_losses.append(avg_val_loss)
    val_accs.append(avg_val_acc)

    # Print metrics per epoch
    print(f"Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f} | "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc:.4f}")

In [ ]:
Epoch 1/10 | Train Loss: 0.0385, Train Acc: 0.9873 | Val Loss: 0.0449, Val Acc: 0.9834
Epoch 2/10 | Train Loss: 0.0382, Train Acc: 0.9874 | Val Loss: 0.0431, Val Acc: 0.9855
Epoch 3/10 | Train Loss: 0.0383, Train Acc: 0.9865 | Val Loss: 0.0436, Val Acc: 0.9836
Epoch 4/10 | Train Loss: 0.0384, Train Acc: 0.9875 | Val Loss: 0.0434, Val Acc: 0.9836
Epoch 5/10 | Train Loss: 0.0384, Train Acc: 0.9877 | Val Loss: 0.0436, Val Acc: 0.9856
Epoch 6/10 | Train Loss: 0.0384, Train Acc: 0.9868 | Val Loss: 0.0432, Val Acc: 0.9851
Epoch 7/10 | Train Loss: 0.0385, Train Acc: 0.9867 | Val Loss: 0.0460, Val Acc: 0.9838
Epoch 8/10 | Train Loss: 0.0385, Train Acc: 0.9872 | Val Loss: 0.0433, Val Acc: 0.9852
Epoch 9/10 | Train Loss: 0.0383, Train Acc: 0.9874 | Val Loss: 0.0447, Val Acc: 0.9836
Epoch 10/10 | Train Loss: 0.0386, Train Acc: 0.9870 | Val Loss: 0.0437, Val Acc: 0.9847

In [ ]:
Testing
Now in this section, we will be testing our model, we will start the code by with torch.no_grad()

In [ ]:
with torch.no_grad():
  total_loss_test = 0
  total_acc_test = 0
  for inputs, labels in test_dataloader:
    prediction = model(inputs).squeeze(1)
    loss = criterion(prediction, labels)

    total_loss_test +=loss.item()
    total_acc_test += ((prediction.round() == labels).sum().item())

print(f"Accuracy Score is: {round((total_acc_test/X_test.shape[0])*100, 2)}%")

In [ ]:
Accuracy Score is: 98.5%